In [1]:
import pandas
import splinter
from bs4 import BeautifulSoup
import time
import selenium

In [2]:
url = 'https://www.drivethrurpg.com/en/browse?ruleSystem=44826'

In [3]:
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# #browser = splinter.Browser()
# browser = selenium.webdriver.Chrome()
# browser.get(url)
# time.sleep(3)
# browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# time.sleep(2)
# browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# soup = BeautifulSoup(browser.page_source, parser="html")
# browser.quit()

In [4]:

browser = splinter.Browser()
browser.visit(url)
time.sleep(3)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
soup = BeautifulSoup(browser.html, parser="html")
browser.quit()

In [5]:
divs = soup.find_all("div", class_="title-strip-carousel-titles-list")
# the newest div is the second title-strip
div = divs[1]

In [6]:
cards = div.find_all("obs-title-strip-card")
for card in cards:
    anchor = card.find("a")
    print(anchor["href"])

/en/product/480148/world-cartographic-world-map-design-templates?src=newest_recent
/en/product/480025/eldritch-maps-the-forest-altar?src=newest_recent
/en/product/479931/europa-theater-bundle?src=newest_recent
/en/product/479847/rangers-in-campaign-6?src=newest_recent
/en/product/479360/la-campana-del-ranger-6?src=newest_recent
/en/product/474088/free-map-dungeon-village?src=newest_recent
/en/product/478987/quasar-trilogy-digital-bundle?src=newest_recent
/en/product/476224/power-outage?src=newest_recent


In [7]:
l = soup.find("ul", class_="list-clean")
for item in l.find_all("li"):
    a = item.find("a", class_="u-dis-block")
    if a != None:
        print(a['href'])

In [8]:
def match_attr(tag):
    return tag.name == "ul" and tag['class'] == ['list-clean'] and not ("_ngcontent-serverapp-c1510817403" in tag.attrs)

In [9]:
def match_obs(tag):
    return tag and tag.name == "obs-result-list" and ("_nghost-serverapp-c1510817403" in tag.attrs)

In [10]:
obs = soup.find(match_obs)
ul = obs.find("ul", class_="list-clean")
lis = ul.find_all("li")
for li in lis:
    a = li.find("a")
    if 'href' in a.attrs:
        print(a['href'])

/en/product/370454/runequest-starter-set
/en/product/475104/going-underground-rivers-of-london-rpg-scenario
/en/product/469816/call-of-cthulhu-arkham
/en/product/260810/call-of-cthulhu-starter-set
/en/product/431991/basic-roleplaying-universal-game-engine
/en/product/462154/heinrich-s-call-of-cthulhu-guide-to-carcosa
/en/product/476836/alone-against-nyarlathotep
/en/product/476571/kane-s-tone
/en/product/446091/japan-empire-of-shadows-a-call-of-cthulhu-sourcebook-for-1920s-imperial-japan
/en/product/476224/power-outage
/en/product/391974/viral-a-modern-call-of-cthulhu-scenario
/en/product/461171/alone-against-the-static
/en/product/480227/the-million-flavoured-ones
/en/product/418884/the-comae-engine
/en/product/476312/in-darkness-we-wait
/en/product/385006/korean
/en/product/468943/book-of-schemes
/en/product/191475/mythras
/en/product/24384/basic-roleplaying
/en/product/386943/korean-the-world-of-cliche
/en/product/365897/korean-100-chronicle-of-100-hours
/en/product/478643/the-oracl